<a href="https://colab.research.google.com/github/emma-thomsen/ADS-assignment-1/blob/main/ADS2022_session1pro_bigDataDask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to big data tools

Python packages like numpy, pandas, sklearn, seaborn etc. make the data manipulation and ML tasks very convenient. For most data analysis tasks, the python pandas package is good enough. You can do all sorts of data manipulation and it is compatible with building ML models.

But, as your data gets bigger, bigger than what you can fit in the RAM, pandas won’t be sufficient. When it comes to working with really large datasets, the run time can become very high due to memory constraints. The standard libraries like pandas and numpy usually work well if the dataset is small enough (upto 2-3 GBs). Unfortunately, these popular libraries were not designed to scale beyond a single machine and given a large dataset to analyze (like 8/16/32  GB or beyond), it would be difficult to process and model it using standard means. 

## Dask

Dask is popularly known as a ‘parallel computing’ python library that has been designed to run across multiple systems. Dask can efficiently perform parallel computations on a single machine using multi-core CPUs. For example, if you have a quad core processor, Dask can effectively use all 4 cores of your system simultaneously for processing. In order to use lesser memory during computations, Dask keeps the complete data on the disk, and uses chunks of data (smaller parts, rather than the whole data) from the disk for processing. During the processing, the intermediate values generated (if any) are discarded as soon as possible, to save the memory consumption.

This way Dask supports the Pandas dataframe and Numpy array data structures to analyze large datasets. Basically, Dask lets you scale pandas and numpy with minimum changes in your code format.

Installation: https://docs.dask.org/en/latest/install.html

Some additional resources for diving deeper into Dask and common operations:

#### Dask documentation: https://docs.dask.org/en/latest/

#### Detailed book/tutorial: https://livebook.manning.com/book/data-science-at-scale-with-python-and-dask/about-this-book/

#### Parallel computing: https://ckyrkou.medium.com/an-introduction-to-parallel-computing-dffa6b79e57c

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from datetime import datetime
from matplotlib import pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

### Dask arrays

A large numpy array is divided into smaller arrays which, when grouped together, form the Dask array. In simple words, Dask arrays are distributed numpy arrays. Every operation on a Dask array triggers operations on the smaller numpy arrays, each using a core on the machine. Thus all available cores are used simultaneously enabling computations on arrays which are larger than the memory size.

<img src="array.png" width="500">


A number of numpy arrays are arranged into grids to form a Dask array. While creating a Dask array, you can specify the chunk size which defines the size of the numpy arrays. For instance, if you have 10 values in an array and you give the chunk size as 5, it will return 2 numpy arrays with 5 values each.

In [ ]:
import dask.array as da
# example
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

This creates a 10000x10000 array of random numbers uniformly distrubuted between 0 and 1. The full object would contain 100M numbers - not impossible, but challenging to handle as a single variable.

Dask would represent it as many numpy arrays of size specified by chunk=1000x1000 (or smaller if the array cannot be divided evenly). 

In this case there are 100 (10x10) numpy arrays of size 1000x1000.

The 'compute()' function changes the result to a numpy array. If you don't call compute, the output remains as a dask object storing the sequency of suggested computations, but not running those just yet ("lazy" approach).

#### Important Note: 

Before calling the compute() function, any dask object remains what is called a Dask 'collection', which essentially stores all the computations called before but not actually performs them. With the compute method, all computations called before are performed simultaneously and the object transform from a dask collection to a concrete value in local memory.

It is thus important to know when and when not to call the compute method. The approach often breaks down if we try to bring the entire dataset as output back to local RAM. So if are dealing with data that exceeds machine RAM, it is better not to obtain the entire data as output. However, we can still call the compute method for obtaining low-memory outputs like computations for selected arrays (or columns in a dataframe) or aggregations.

In [ ]:
x.compute()

array([[0.03053164, 0.76191926, 0.76954501, ..., 0.38169206, 0.74321877,
        0.20074126],
       [0.69330788, 0.84396481, 0.83420801, ..., 0.92703938, 0.74347352,
        0.8409397 ],
       [0.88385563, 0.4338358 , 0.16777118, ..., 0.75499232, 0.27233158,
        0.62254311],
       ...,
       [0.78420229, 0.83436419, 0.73316962, ..., 0.42095732, 0.65973039,
        0.53165075],
       [0.99727884, 0.87177474, 0.87050893, ..., 0.40681534, 0.19399229,
        0.51213468],
       [0.34421772, 0.35374967, 0.62969886, ..., 0.83079682, 0.42504504,
        0.59726502]])

### Persist data in memory
If you know you have the available RAM for your dataset then you can persist data in memory. Essentially this operation temporarily saves the data in your computer memory if the size of the data does not exceed your RAM.

This allows future computations to be much faster. 

Let's check how much time does it take to calculate sum of the above array

In [ ]:
# calculate sum and call compute to print the output
%time x.sum().compute()

CPU times: user 1.67 s, sys: 155 ms, total: 1.83 s
Wall time: 1.01 s


50001993.37288932

In [ ]:
# now let's try the persist() function and then check the time
y = x.persist()
%time y.sum().compute()

CPU times: user 151 ms, sys: 10.9 ms, total: 162 ms
Wall time: 94.7 ms


50001993.37288932

The computation time almost reduces to 1/10th of the original time after calling the persist() function

### Dask dataframes

We saw that multiple numpy arrays are grouped together to form a Dask array. Similar to a Dask array, a Dask dataframe consists of multiple smaller pandas dataframes which are stored in disk/cluster as a single dask object. These Pandas DataFrames may live on disk for computing on a single machine, or on many different machines in a cluster. One Dask DataFrame operation triggers many operations on the constituent Pandas DataFrames.

A large pandas dataframe splits row-wise to form multiple smaller dataframes. These smaller dataframes are present on a disk of a single machine, or multiple machines (thus allowing to store datasets of size larger than the memory). Each computation on a Dask dataframe parallelizes operations over the different chunks on the dataframe.


<img src="dataframe.png" width="300">


### Common uses:

Dask DataFrame is used in situations where Pandas is commonly needed, usually when Pandas fails due to data size or computation speed.

1. Manipulating large datasets, even when those datasets don’t fit in memory
2. Accelerating long computations by using many cores
3. Distributed computing on large datasets with standard Pandas operations like groupby, join, and time series computations

The APIs offered by the Dask dataframe are very similar in syntax to that of the pandas dataframe.

#### Now, let’s perform some basic operations on Dask dataframes

Let's download 2020 TLC yellow taxi data for first few months. Generally, TLC data contains millions of records even for a single month and pandas usually has a hard time reading and performing computations on it. Anyway, pandas either altogether fails to load data for multiple months or takes a huge time reading it.

In [ ]:
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv > yellow_tripdata_2020-01.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-02.csv > yellow_tripdata_2020-02.csv
# !curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv > yellow_tripdata_2020-03.csv
# !curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-04.csv > yellow_tripdata_2020-04.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  566M  100  566M    0     0  43.1M      0  0:00:13  0:00:13 --:--:-- 47.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  557M  100  557M    0     0  43.7M      0  0:00:12  0:00:12 --:--:-- 47.5M


Let's first compare the time taken to read a single month of data by dask and pandas. 

In [ ]:
# read file: 'read_csv()' works just like pandas
%time df = dd.read_csv('yellow_tripdata_2020-01.csv')
#df.head()

CPU times: user 23.9 ms, sys: 7.01 ms, total: 30.9 ms
Wall time: 36.4 ms


In [ ]:
# let's check time with pandas 
%time df1 = pd.read_csv('yellow_tripdata_2020-01.csv')
df1.head()

CPU times: user 10.9 s, sys: 1.71 s, total: 12.6 s
Wall time: 14.5 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


We can see the huge difference between dask and pandas just for reading a single month of data (around 6M rows). Dask is ~150 times faster than pandas here!

Let's read data for all months now. Notice we specify the variable number in the address as * to read all months data. We can also specify the dtypes of the variables as below.

In [ ]:
df = dd.read_csv('yellow_tripdata_2020-*.csv', dtype={'trip_distance': float,
                        'total_amount': float, 'tolls_amount':float, 'RatecodeID': float, 'VendorID': float, 
                                                     'passenger_count': float, 'payment_type':float, 
                                                     'PULocationID':int, 'DOLocationID':int})
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


#### Notice:
The .head() function automatically converts the dask dataframe to pandas dataframe. Unlike Pandas, Dask DataFrames are lazy and so no data is printed. Let's try printing a dask dataframe.

In [ ]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=20,,,,,,,,,,,,,,,,,,
,float64,object,object,float64,float64,float64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Observe that in a dask dataframe, we don't see the inherent values of data but instead only get information about columns and their dtypes

### Common pandas operations that can be used in dask

As mentioned before, The operations offered by the Dask dataframe are very similar to that of pandas.

In [ ]:
#dtypes operation for checking column dtypes
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [ ]:
# dropna and droplicates work the same way here
# one difference: some arguments like 'inplace' do not work

# lets drop rows where PULocation and DOLocation are not present
df = df.dropna(subset=['PULocationID', 'DOLocationID'])

# drop duplicate rows
df = df.drop_duplicates()

In [ ]:
# now let's try making a new column:travel time
# first need to make pickup and dropoff time columns as datetime - like pandas, 'to_datetime()' operation works here

#convert columns to datetime dtype
df['tpep_pickup_datetime'] = dd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = dd.to_datetime(df['tpep_dropoff_datetime'])

# make a travel time column (minutes)
df['travel_time'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['travel_time'] = (df['travel_time'].dt.seconds)/60

In [ ]:
# now let's try calculating some descriptive statistics for trip distance and total amount

df[['trip_distance', 'total_amount']].describe()

,trip_distance,total_amount
npartitions=1,,
,float64,float64
,...,...


Remember from before that dask dataframe themselves don't print out enough info. So in order to print required descriptive stats like pandas, we need to convert the above dataframe to pandas dataframe. 

This can be done by compute()

In [ ]:
# this could take some time
df[['trip_distance', 'total_amount']].describe().compute()

KeyboardInterrupt: ignored

In [ ]:
# groupby operation - works the same as pandas

%time df.groupby('PULocationID').count()[['DOLocationID']].compute()

In [ ]:
# calculate mean trip distance and travel time per pickup location

%time df[['PULocationID', 'trip_distance', 'travel_time']].groupby('PULocationID').mean().compute()

NameError: ignored

Notice that converting to calculating stats and converting to pandas take quite some time. This is because once we call compute(), all operations done above are executed all at once.

One way to reduce this time is by assigning some RAM of your machine to the data. This can be done by persist() operation described above.

#### Important note:

Persist method turns lazy Dask collections into Dask collections with the same metadata, but now with their results fully computed or actively computing in the background. For example a dataframe built up from many lazy calls will now be a new dataframe of the same shape, dtype, chunks, etc., but now with all of those previously lazy tasks either computed in memory as many small dataframes (in the single-machine case) or asynchronously running in the background on a cluster (in the distributed case).

Generally, it is better to run persist method regularly among computations if we are dealing with large number of computations in our work (provided we have sufficient memory on your machine). A large number of computations executed at once may overload the memory and task could fail in that case. 

Now let's try reducing the data size by keeping only the columns we need for further analysis. We can then run persist method to allocate some RAM to the resulting data and perform all computations till now.

In [ ]:
df.columns

In [ ]:
# select few columns that we need for further
df = df[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 
        'trip_distance', 'PULocationID', 'DOLocationID', 'total_amount', 'travel_time']]

In [ ]:
# persist operation to assign some RAM to data
df = df.persist()

Note: 'df' still remains a dask object after calling persist method, but all operations done before this point would be computed after calling persist. One further advantage of persist is the time in further computations would be significantly reduced going furhter on. Essentially, persist stores the updated data as a dask object after doing all computations on the data.

In [ ]:
type(df)

Let's now try some operations. Notice the computation time would be significantly reduced

In [ ]:
# calculate mean trip distance and travel time per pickup location

%time df[['PULocationID', 'trip_distance', 'travel_time']].groupby('PULocationID').mean().compute()

In [ ]:
# Let's try the merge operation

# read taxi zone file
zips = gpd.read_file('https://github.com/CUSP2022ADS/Data/raw/main/taxizone.geojson')
zips = zips[['location_id', 'borough', 'shape_area', 'zone']]
zips['location_id'] = pd.to_numeric(zips['location_id'])
zips.head()

In [ ]:
# merge df with zips file

df = df.merge(zips, how='left', left_on='PULocationID', right_on='location_id')

In [ ]:
# we may also use matplotlib functions with dask 

tripByBoro = df.groupby('borough').count()[['DOLocationID']].compute()
plt.bar(tripByBoro.index, tripByBoro.DOLocationID.values)
plt.ylabel('number of trips')

### HW task 1

Compute and Plot 

1. Plot average number of trips as bar plots by day of week
2. Plot average total fare amount by hour of pick up time
3. Average speed by pick up hour (average speed should be calculated as total distance traveled by hour/total travel time by hour. Plot as barplot.
4. Report top 5 and bottom 5 pickup locations in terms of a) total distance, b) average speed

In [1]:
df.head()

NameError: ignored

In [ ]:
# problem 1
df['DOW'] = df.datetime.dayofweek
df1 = df[['DOW', 'tpep_pickup_datetime']].groupby('DOW').count().compute()

ax = df1.plot.bar(x=df1['DOW'], y=df1['tpep_pickup_datetime'])

In [ ]:
# problem 2
df['HOUR'] = df.datetime.hour
df2 = df[['HOUR', 'total_amount']].groupby('HOUR').mean().compute()

ax = df2.plot.bar(x=df2['HOUR'], y=df2['total_amount'])

In [ ]:
#problem 3
df['SPEED'] = (df['trip_distance'])/(df['travel_time'])

df3 = df[['HOUR', 'SPEED']].groupby('HOUR').mean().compute()

ax = df2.plot.bar(x=df2['HOUR'], y=df2['SPEED'])

## Machine learning operations with Dask

### Dask-ML

Dask-ML provides some ensemble methods that are tailored to dask.array’s and dask.dataframe’s blocked structure. The basic idea is to fit a copy of some sub-estimator to each block (or partition) of the dask Array or DataFrame. Becuase each block fits in memory, the sub-estimator only needs to handle in-memory data structures like a NumPy array or pandas DataFrame. It also will be relatively fast, since each block fits in memory and we won’t need to move large amounts of data between workers on a cluster. We end up with an ensemble of models: one per block in the training dataset.

At prediction time, we combine the results from all the models in the ensemble. For regression problems, this means averaging the predictions from each sub-estimator. For classification problems, each sub-estimator votes and the results are combined.

#### Note: 
It’s crucially important that the distribution of values in your dataset be relatively uniform across partitions. Otherwise the parameters learned on any given partition would vary widely and might not be well applicable to the dataset as a whole.

In [ ]:
import dask_ml.datasets
import dask_ml.ensemble

## try linear regression 
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import train_test_split

X, y = dask_ml.datasets.make_regression(n_samples=1_000_000,
                                        chunks=100_000,
                                        n_features=20)
X

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

In [ ]:
lr.predict(X)[:5].compute()

In [ ]:
(lr.intercept_, lr.coef_)

In [ ]:
lr.score(X, y)

In [ ]:
# try K-means clustering
import dask_ml.cluster

X, y = dask_ml.datasets.make_blobs(n_samples=1000000,
                                   chunks=100000,
                                   random_state=0,
                                   centers=3)
X = X.persist()
X

In [ ]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(X)

In [ ]:
km.labels_[::1000]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[::1000, 0], X[::1000, 1], marker='.', c=km.labels_[::1000],
           cmap='viridis', alpha=0.25);

Note: dask_ml operations process dask objects instead of numpy arrays or pandas dataframe. Unless we want to print results as numpy array or pandas dataframe, we don't need to call compute. This makes dask_ml much faster than other libraries like sklearn. 

## Dask_ml for learning average speed in the taxi data

Implement a linear regression of the travel time vs the trip distance

In [ ]:
# do some filtering to remove outliers and wrong values
df = df[(df.trip_distance > 0) & (df.trip_distance < 50)]
df = df[(df.total_amount > 0) & (df.total_amount < 300)]
df = df[(df.travel_time > 0) & (df.travel_time < 200)]

In [ ]:
lr_taxi = LinearRegression(fit_intercept=False)
lr_taxi.fit(df[['trip_distance']].to_dask_array(),df[['travel_time']].to_dask_array())

In [ ]:
lr_taxi.coef_ #slope coefficient (min per mile)

In [ ]:
dist_sample=df[['trip_distance']].to_dask_array().compute()[:10] #take first 10 distances as a sample

In [ ]:
#return the predicted vs actual time for the selected sample

pd.DataFrame({'distance':dist_sample.flatten(), 'predicted time':lr_taxi.predict(dist_sample), 'actual time': df[['travel_time']].to_dask_array().compute()[:10].flatten()})

#### Important note:

Recently, there seem to be some issues with dask_ml model when we train models on custom data and large dask dataframes. dask_ml library is undergoing changes constantly has a lot of bugs and seems to be getting updated frequently. Hence, we'll not demonstrate any examples on real world data with dask_ml.

Check dask_ml's github repo to follow through regular updates: https://github.com/dask/dask-ml

Documentation: https://ml.dask.org/

next we'll see how scikit-learn can be scaled for big-data modeling

## Scalable Scikit-Learn

Alternatively, we can use scikit-learn for training models on medium sized data where pandas would usually fail or would take a huge amount of time.

Scikit-learn uses joblib for single-machine parallelism. This lets you train most estimators (anything that accepts an n_jobs parameter) using all the cores of your laptop or workstation. Alternatively, Scikit-Learn can use Dask for parallelism. This lets you train those estimators using all the cores of your cluster without significantly changing your code.

This is most useful for training large models on medium-sized datasets. You may have a large model when searching over many hyper-parameters, or when using an ensemble method with many individual estimators. For too small datasets, training times will typically be small enough that cluster-wide parallelism isn’t helpful. For too large datasets (larger than a single machine’s memory), the scikit-learn estimators may not be able to cope.

In [ ]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import numpy as np

Let's now use skearn to build a simple linear model with Lasso regularization with trip cost amount as a target variable and using just trip distance as a regressor. We'll use GridSearchCV for tuning the hyperparameter (alpha) value.

In [ ]:
# do some filtering to remove outliers and wrong values
df = df[(df.trip_distance > 0) & (df.trip_distance < 50)]
df = df[(df.total_amount > 0) & (df.total_amount < 300)]
df = df[(df.travel_time > 0) & (df.travel_time < 200)]

In [ ]:
# X as features, y as target (note: we need to call compute method, as sklearn only takes numpy arrays)
X = df[['trip_distance','travel_time']].values.compute()
y = df.total_amount.values.compute()

Split the data into training and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2002)

#### Create Scikit-Learn Pipeline

Pipeline consist of all the models (or intermediate steps) that we wish to train/perform on the data

In [ ]:
# make a pipeline - specify the model
pipeline = Pipeline([
    ('lasso', Lasso(fit_intercept=True))])

# we can specify parameters here we want to optimize
# notice the format - {'model__parameter': range}, this format should follow for all parameters
parameters = {'lasso__alpha': 2.0**(np.arange(-10, 10, 1))}

# call the gridsearch method and fit
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=3, refit=True)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
# r2 score
grid_search.score(X_test, y_test)

Note: results from scalable scikit learn and dask ml could slightly differ from the simple regression (done with whole data at once) as those operations combine results from different chunks and average them.

### Extra credit: Homework task 2



With trip cost amount as a target variable and using trip distance, travel time and PULocationID as predictors, build a Random Forest Regression with 10 trees (n_estimators) using the sklearn pipeline to select the hyperparameter 'max_depth' with values from 2 to 5.

Please review the Random forest documentation here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Use 75:25 train test split as above, use the training data for GridSearchCV with three-fold cross-validation (cv=3). Report the best max_depth as well as the R2 on test data.

In [ ]:
# prepare X as features and y as target
X = df[['trip_distance','travel_time']].values.compute()
y = df.total_amount.values.compute()

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2002)

In [ ]:
# create pipeline and fit the model
pipeline = Pipeline([
    ('random forrest', RandomForrestRegressor(fit_intercept=True))])

parameters = {max_depth : [2:5], default = None}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=3, refit=True)

grid_search.fit(X_train, y_train)

In [ ]:
# report test accuracy
grid_search.score(X_test, y_test)